In [1]:
import pandas as pd
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials 
cid ="5cf7fd8a74434b4aa2243cd27dc0e490" 
secret = "033870832e60454b8c23e5e7a56b4280" 
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

artist_name = []
track_name = []
popularity = []
track_id = []
for i in range(0,1000,50): 
    track_results = sp.search(q='year:2018', type='track', limit=50,offset=i) 
    for i, t in enumerate(track_results['tracks']['items']):
           artist_name.append(t['artists'][0]['name']) 
           track_name.append(t['name']) 
           track_id.append(t['id']) 
           popularity.append(t['popularity'])


df_tracks = pd.DataFrame({'popularity':popularity,'artist_name':artist_name,'track_name':track_name,'track_id':track_id})
# df_tracks = df_tracks.sort_values('popularity', ascending=False)
print(df_tracks.shape)
df_tracks.head()

grouped = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped[grouped > 1].count()

(1000, 4)


13

In [2]:
df_tracks.drop_duplicates(subset=['artist_name','track_name'], inplace=True)

In [3]:
grouped_after_dropping = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped_after_dropping[grouped_after_dropping > 1].count()

0

In [4]:
df_tracks.shape

(987, 4)

In [5]:
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

Number of tracks where no audio features were available: 0


In [6]:
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

Shape of the dataset: (987, 18)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.614,0.6800,6,-4.580,1,0.0289,0.3690,0.000002,0.115,0.707,149.959,audio_features,6foY66mWZN0pSRjZ408c00,spotify:track:6foY66mWZN0pSRjZ408c00,https://api.spotify.com/v1/tracks/6foY66mWZN0p...,https://api.spotify.com/v1/audio-analysis/6foY...,234347,4
1,0.314,0.0855,9,-15.775,1,0.0342,0.9690,0.795000,0.160,0.161,69.893,audio_features,0UsmyJDsst2xhX1ZiFF3JW,spotify:track:0UsmyJDsst2xhX1ZiFF3JW,https://api.spotify.com/v1/tracks/0UsmyJDsst2x...,https://api.spotify.com/v1/audio-analysis/0Usm...,74302,3
2,0.515,0.4790,3,-7.458,1,0.0261,0.5440,0.005980,0.191,0.284,88.964,audio_features,0rKtyWc8bvkriBthvHKY8d,spotify:track:0rKtyWc8bvkriBthvHKY8d,https://api.spotify.com/v1/tracks/0rKtyWc8bvkr...,https://api.spotify.com/v1/audio-analysis/0rKt...,209274,4
3,0.684,0.6380,9,-9.025,0,0.0312,0.3650,0.020700,0.119,0.810,129.959,audio_features,4fbwTO3DJ2qryMddov9RbB,spotify:track:4fbwTO3DJ2qryMddov9RbB,https://api.spotify.com/v1/tracks/4fbwTO3DJ2qr...,https://api.spotify.com/v1/audio-analysis/4fbw...,226773,4
4,0.897,0.6620,1,-6.903,0,0.2920,0.0852,0.000000,0.534,0.389,112.511,audio_features,78QR3Wp35dqAhFEc2qAGjE,spotify:track:78QR3Wp35dqAhFEc2qAGjE,https://api.spotify.com/v1/tracks/78QR3Wp35dqA...,https://api.spotify.com/v1/audio-analysis/78QR...,145543,4


In [7]:
df_audio_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 987 entries, 0 to 986
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      987 non-null    float64
 1   energy            987 non-null    float64
 2   key               987 non-null    int64  
 3   loudness          987 non-null    float64
 4   mode              987 non-null    int64  
 5   speechiness       987 non-null    float64
 6   acousticness      987 non-null    float64
 7   instrumentalness  987 non-null    float64
 8   liveness          987 non-null    float64
 9   valence           987 non-null    float64
 10  tempo             987 non-null    float64
 11  type              987 non-null    object 
 12  id                987 non-null    object 
 13  uri               987 non-null    object 
 14  track_href        987 non-null    object 
 15  analysis_url      987 non-null    object 
 16  duration_ms       987 non-null    int64  
 1

In [8]:
columns_to_drop = ['analysis_url','track_href','type','uri']
df_audio_features.drop(columns_to_drop, axis=1,inplace=True)

df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)

df_audio_features.shape

(987, 14)

In [9]:
df = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df.head()

Shape of the dataset: (987, 14)


,popularity,artist_name,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,81,Morgan Wallen,Whiskey Glasses,6foY66mWZN0pSRjZ408c00,0.614,0.6800,6,-4.580,1,0.0289,0.3690,0.000002,0.115,0.707,149.959,234347,4
1,24,Schoolgirl Byebye,"Year,2015",0UsmyJDsst2xhX1ZiFF3JW,0.314,0.0855,9,-15.775,1,0.0342,0.9690,0.795000,0.160,0.161,69.893,74302,3
2,85,Joji,SLOW DANCING IN THE DARK,0rKtyWc8bvkriBthvHKY8d,0.515,0.4790,3,-7.458,1,0.0261,0.5440,0.005980,0.191,0.284,88.964,209274,4
3,48,Fleetwood Mac,Rhiannon (Will You Ever Win) - 2018 Remaster,4fbwTO3DJ2qryMddov9RbB,0.684,0.6380,9,-9.025,0,0.0312,0.3650,0.020700,0.119,0.810,129.959,226773,4
4,84,Lil Baby,Drip Too Hard (Lil Baby & Gunna),78QR3Wp35dqAhFEc2qAGjE,0.897,0.6620,1,-6.903,0,0.2920,0.0852,0.000000,0.534,0.389,112.511,145543,4


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 987 entries, 0 to 986
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   popularity        987 non-null    int64  
 1   artist_name       987 non-null    object 
 2   track_name        987 non-null    object 
 3   track_id          987 non-null    object 
 4   danceability      987 non-null    float64
 5   energy            987 non-null    float64
 6   key               987 non-null    int64  
 7   loudness          987 non-null    float64
 8   mode              987 non-null    int64  
 9   speechiness       987 non-null    float64
 10  acousticness      987 non-null    float64
 11  instrumentalness  987 non-null    float64
 12  liveness          987 non-null    float64
 13  valence           987 non-null    float64
 14  tempo             987 non-null    float64
 15  duration_ms       987 non-null    int64  
 16  time_signature    987 non-null    int64  
dt

In [11]:
df[df.duplicated(subset=['artist_name','track_name'],keep=False)]

,popularity,artist_name,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature


In [12]:
df.to_csv('SpotifyAudioFeatures08082018.csv')